In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

# Visualization of RICE-N and RL training


In [ ]:
import os, glob, sys, numpy as np, scipy as sp, sklearn as skl

In [ ]:
if "_ROOT" not in globals():
    _ROOT = os.getcwd()
    print(f"Set _ROOT = {_ROOT}")
else: 
    print(f"Already set: _ROOT = {_ROOT}")

## Save or load from previous training results

This section is for saving and loading the results of training (not the trainer itself).

In [ ]:
from opt_helper import save, load

To save the output timeseries: 

In [ ]:
# [uncomment below to save]
# save({"nego_off":gpu_nego_off_ts, "nego_on":gpu_nego_on_ts}, "filename.pkl")

To load the output timeseries:

In [ ]:
# [uncomment below to load]
dict_ts = load("example_data/example.pkl")
nego_off_ts, nego_on_ts = dict_ts["nego_off"], dict_ts["nego_on"]

In [ ]:
for key, value in nego_off_ts.items(): 
    print(f"{key:40} {value.shape}")

## Plot training procedures

One may want to plot the some metrics such as `mean reward` which are logged during the training procedure.

```python
metrics = ['Iterations Completed',
 'VF loss coefficient',
 'Entropy coefficient',
 'Total loss',
 'Policy loss',
 'Value function loss',
 'Mean rewards',
 'Max. rewards',
 'Min. rewards',
 'Mean value function',
 'Mean advantages',
 'Mean (norm.) advantages',
 'Mean (discounted) returns',
 'Mean normalized returns',
 'Mean entropy',
 'Variance explained by the value function',
 'Gradient norm',
 'Learning rate',
 'Mean episodic reward',
 'Mean policy eval time per iter (ms)',
 'Mean action sample time per iter (ms)',
 'Mean env. step time per iter (ms)',
 'Mean training time per iter (ms)',
 'Mean total time per iter (ms)',
 'Mean steps per sec (policy eval)',
 'Mean steps per sec (action sample)',
 'Mean steps per sec (env. step)',
 'Mean steps per sec (training time)',
 'Mean steps per sec (total)'
 ]
```

To check out the logged submissions, please run the following block.

In [ ]:
from glob import glob
submission_zip_files = glob(os.path.join(_ROOT,"Submissions/*.zip"))

If previous trainings are finished and logged properly, this should give a list of `*.zip` files where the logs are included. 

We picked one of the submissions and the metric `Mean episodic reward` as an example, please check the code below.

In [ ]:
from opt_helper import get_training_curve, plot_training_curve

### WarpDrive version

In [ ]:
# TBC

### RLLib (CPU) version

In [ ]:
log_zip = submission_zip_files[1]
plot_training_curve(None, 'Mean episodic reward', log_zip)

# to check the raw logging dictionary, uncomment below
# logs = get_training_curve(log_zip)
# logs

## Plot results

In [ ]:
from scripts.desired_outputs import desired_outputs

One may want to check the performance of the agents by plotting graphs. Below, we list all the logged variables. One may change the ``desired_outputs.py`` to add more variables of interest.

```python
desired_outputs = ['global_temperature', 
  'global_carbon_mass', 
  'capital_all_regions', 
  'labor_all_regions', 
  'production_factor_all_regions', 
  'intensity_all_regions', 
  'global_exogenous_emissions', 
  'global_land_emissions', 
  'timestep', 
  'activity_timestep', 
  'capital_depreciation_all_regions', 
  'savings_all_regions', 
  'mitigation_rate_all_regions', 
  'max_export_limit_all_regions', 
  'mitigation_cost_all_regions', 
  'damages_all_regions', 
  'abatement_cost_all_regions', 
  'utility_all_regions', 
  'social_welfare_all_regions', 
  'reward_all_regions', 
  'consumption_all_regions', 
  'current_balance_all_regions', 
  'gross_output_all_regions', 
  'investment_all_regions', 
  'production_all_regions', 
  'tariffs', 
  'future_tariffs', 
  'scaled_imports', 
  'desired_imports', 
  'tariffed_imports', 
  'stage', 
  'minimum_mitigation_rate_all_regions', 
  'promised_mitigation_rate', 
  'requested_mitigation_rate', 
  'proposal_decisions',
  'global_consumption',
  'global_production']
```

In [ ]:
from opt_helper import plot_result

`plot_result()` plots the time series of logged variables.

```python
plot_result(variables, nego_off, nego_on, k)
```
* ``variables`` can be either a single variable of interest or a list of variable names from the above list. 
* The ``nego_off_ts`` and ``nego_on_ts`` are the logged time series for these variables, with and without negotiation. 
* ``k`` represents the dimension of the variable of interest ( it should be ``0`` by default for most situations).

Here's an example of plotting a single variable of interest.

In [ ]:
plot_result("global_temperature", 
  nego_off=nego_off_ts, # change it to cpu_nego_off_ts if using CPU
  nego_on=nego_on_ts, 
  k=0)

Here's an example of plotting a list of variables.

In [ ]:
plot_result(desired_outputs[0:3], # truncated for demonstration purposes
  nego_off=nego_off_ts, 
  nego_on=nego_on_ts, 
  k=0)

If one only want to plot negotiation-off plots, feel free to set `nego_on=None`. 

In [ ]:
plot_result(desired_outputs[0:3], # truncated for demonstration purposes
  nego_off=nego_off_ts, 
  nego_on=None, 
  k=0)

## Plot region data using a grid plot

In [ ]:
from opt_helper import make_grid_plot

In [ ]:
feature_name = "labor_all_regions"
feature_label = feature_name.replace("_", " ").title() + " - atmosphere layer"
make_grid_plot(
    nego_off_ts[feature_name], 
    xlabel="Year", 
    ylabel=feature_label, 
    feature_label=feature_label, 
    cols=3, 
    fig_scale=3
);

## Plot multiple time series data with mean and spread around the mean

In [ ]:
from opt_helper import plot_fig_with_bounds

Generating some example perturbed data around the original data.

In [ ]:
from copy import deepcopy 
perturbed_nego_on_ts = [
    deepcopy(nego_on_ts),
    deepcopy(nego_on_ts),
    deepcopy(nego_on_ts),
]

for key, ts in nego_on_ts.items():
    perturbed_nego_on_ts[0][key] = ts
    perturbed_nego_on_ts[1][key] = ts + 0.1 * ts
    perturbed_nego_on_ts[2][key] = ts - 0.1 * ts


perturbed_nego_off_ts = [
    deepcopy(nego_off_ts),
    deepcopy(nego_off_ts),
    deepcopy(nego_off_ts),
]

for key, ts in nego_off_ts.items():
    perturbed_nego_off_ts[0][key] = ts
    perturbed_nego_off_ts[1][key] = ts + 0.1 * ts
    perturbed_nego_off_ts[2][key] = ts - 0.1 * ts

In [ ]:
feature_name ="labor_all_regions"

plot_fig_with_bounds(
    feature_name, # variable,
    "y_label", # y_label,
    list_of_dict_off=perturbed_nego_off_ts,
    list_of_dict_on=perturbed_nego_on_ts,
    title=None,
    idx=0,
    x_label="year",
    skips=3,
    line_colors=["#0868ac", "#7e0018"],
    region_colors=["#7bccc4", "#ffac3b"],
    start=2020,
    alpha=0.5,
    is_grid=True,
    is_save=True,
    delta=5,
)

## Aggregate simulation statistics to visualize

In [ ]:
for k, v in nego_off_ts.items(): 
    print(f"{k:40}{v.shape}")

In [ ]:
feature_name = "mitigation_rate_all_regions"
feature_label = feature_name.replace("_", " ").title()
make_grid_plot(
    nego_on_ts[feature_name],
    feature_label=feature_label, 
    xlabel="Step",
    ylabel=feature_label,
    cols=4,
    fig_scale=3,
);

In [ ]:
feature_name = "savings_all_regions"
feature_label = feature_name.replace("_", " ").title()
make_grid_plot(
    nego_on_ts[feature_name],
    feature_label=feature_label, 
    xlabel="Step",
    ylabel=feature_label,
    cols=4,
    fig_scale=3,
);

## Cluster regions

In [ ]:
feature_name = "savings_all_regions"
lo_savings, med_savings, hi_savings = [], [], []
steps, n_regions = nego_on_ts[feature_name].shape
for region_j in range(n_regions):
    mean_region_j = np.mean(nego_on_ts[feature_name][:, region_j])
    print(f"{region_j}: {mean_region_j:.2f}")
    if mean_region_j < 0.25: 
        lo_savings.append(region_j)
    elif 0.25 < mean_region_j < 0.35:
        med_savings.append(region_j)
    else:
        hi_savings.append(region_j)

In [ ]:
lo_savings, med_savings, hi_savings

In [ ]:
from opt_helper import make_aggregate_data_across_three_clusters

In [ ]:
aggregate_nego_on_ts = make_aggregate_data_across_three_clusters(
    nego_on_ts, 
    lo_savings, 
    med_savings, 
    hi_savings
)

In [ ]:
feature_name = "savings_all_regions"
feature_label = feature_name.replace("_", " ").title() + " - aggregate"
make_grid_plot(
    aggregate_nego_on_ts[feature_name],
    feature_label=feature_label, 
    xlabel="Step",
    ylabel=feature_label,
    cols=4,
    fig_scale=3,
);

## Correlation between clusters and feature of those cluster members?

In [ ]:
from opt_helper import compute_correlation_across_groups

In [ ]:
# Define the groups of regions that are aggregated.
# Note: these shoudl be the same groups that were used to generate the aggregate statistics.
groups = (lo_savings, med_savings, hi_savings)

# Defint the X variable
x_feature_name = "savings_all_regions"
aggregate_stats_across_groups = aggregate_nego_on_ts[x_feature_name]

# Define the y data
data_ts = nego_on_ts

# Only compute correlations for the y variables that are region-specific time-series.
y_features_names = [
    i for i in data_ts.keys() if "all_regions" in i
]

for y_feature_name in y_features_names: 
    r2 = compute_correlation_across_groups(aggregate_stats_across_groups, 
        data_ts, 
        y_feature_name, 
        do_plot=False
    )

    print(f"{x_feature_name:35} vs {y_feature_name:35} r2 = {r2:5.2f}")